<a href="https://colab.research.google.com/github/arimbawa/Aplikasi-IoT/blob/main/Minggu_ke_7_Keamanan_IoT_(HTTP_%26_MQTT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Minggu ke-7: Keamanan IoT (HTTP & MQTT)**
**Tujuan Pembelajaran:**

1. Mahasiswa memahami risiko keamanan pada komunikasi IoT.
2. Mahasiswa dapat menerapkan HTTPS (SSL/TLS) pada komunikasi HTTP.
3. Mahasiswa dapat mengamankan MQTT dengan username-password authentication.
4. Mahasiswa mengetahui praktik dasar secure coding dalam aplikasi IoT berbasis Flask & MQTT.

# Praktik di Google Colab

# 1. Install modul

In [ ]:
!pip install flask pyngrok paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.3 MB/s eta 0:00:00


# 2. Flask dengan Token Authentication

In [3]:
# Konfigurasi Ngrok dengan token Agus
from pyngrok import ngrok

NGROK_TOKEN = "xxxxxxxxxxxx"
ngrok.set_auth_token(NGROK_TOKEN)

In [4]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# Token sederhana
API_TOKEN = "iotsecure123"

@app.route('/secure-data', methods=['POST'])
def secure_data():
    token = request.headers.get("Authorization")
    if token != f"Bearer {API_TOKEN}":
        return jsonify({"error": "Unauthorized"}), 401

    data = request.json
    return jsonify({"status": "success", "data_received": data}), 200

# Jalankan ngrok
public_url = ngrok.connect(5000)
print(" * Ngrok URL (secure):", public_url)

app.run(port=5000)

 * Ngrok URL (secure): NgrokTunnel: "https://fe8b4251e388.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Sep/2025 03:42:02] "POST /secure-data HTTP/1.1" 200 -


**Cara uji:**

In [ ]:
import requests

url = "<NGROK_URL>/secure-data"
headers = {"Authorization": "Bearer iotsecure123"}
data = {"temperature": 25, "humidity": 60}

r = requests.post(url, json=data, headers=headers)
print(r.text)

Jika tanpa header Authorization → 401 Unauthorized.

# 3. MQTT dengan Username & Password (HiveMQ broker publik)

In [5]:
import paho.mqtt.client as mqtt
import random, time, json

# HiveMQ public broker with auth (contoh)
BROKER = "broker.hivemq.com"
PORT = 1883
TOPIC = "unram/iot/secure"
USERNAME = "iot_user"
PASSWORD = "iot_pass"

def on_connect(client, userdata, flags, rc):
    print("Connected with result code", rc)

def on_message(client, userdata, msg):
    print(f"Message received: {msg.payload.decode()}")

client = mqtt.Client()
client.username_pw_set(USERNAME, PASSWORD)  # Auth
client.on_connect = on_connect
client.on_message = on_message

client.connect(BROKER, PORT, 60)
client.loop_start()

# Publish data sensor dengan autentikasi
for i in range(5):
    payload = {"temperature": random.randint(20,30),
               "humidity": random.randint(60,80)}
    client.publish(TOPIC, json.dumps(payload))
    print("Published:", payload)
    time.sleep(2)

/tmp/ipython-input-1815004766.py:17: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Published: {'temperature': 22, 'humidity': 73}
Published: {'temperature': 30, 'humidity': 79}
Published: {'temperature': 30, 'humidity': 60}
Connected with result code 0
Published: {'temperature': 26, 'humidity': 60}
Published: {'temperature': 20, 'humidity': 77}


# Tugas Minggu ke-7

1. Ubah server Flask agar menggunakan HTTPS (bisa dengan sertifikat self-signed atau pyOpenSSL).
2. Buat client IoT yang hanya bisa mengirim data dengan API key valid.
3. Konfigurasi MQTT client agar gagal connect jika username/password salah.
4. Buat laporan singkat: bandingkan komunikasi tanpa keamanan vs dengan keamanan (HTTP/MQTT).T